In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy
import scipy.io as sio
import scipy.io.wavfile

In [ ]:
# Load code challenge file
challenge = scipy.io.loadmat('filtering_codeChallenge.mat')

In [ ]:
x = np.squeeze(challenge['x'])
y = np.squeeze(challenge['y'])

fs = challenge['fs'][0][0] #sampling frequency
npnts = len(y)
dt = 1/fs
time_vec = np.arange(0,npnts)/fs

# Plot data from mat file
plt.subplot(211) 

plt.plot(time_vec, x, label='x = original')
plt.plot(time_vec, y, label='y = filtered')
plt.legend()
plt.title('Time domain')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

# Look at frequency plots of both 

# Get fourier transform
pwr_x = np.abs(scipy.fft.fft(x)/npnts)**2
pwr_y = np.abs(scipy.fft.fft(y)/npnts)**2
hz = np.linspace(0, fs, npnts)

plt.subplot(212)
plt.plot(hz,pwr_x,label='original')
plt.plot(hz,pwr_y,label='filtered')
plt.title('Frequency domain')
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.subplots_adjust(hspace=1)
plt.xlim([0,40])


In [ ]:
# Looks like the filtered signal has two ranges which are not attenuated
# First band is [5,17]
# Second band is [25,33]

# Strategy:
# Try bandpass filtering for [5,17] first, then  try [25,33]
# Make sure to look at filter spectral response before applying filter

# Define first filter parameter
lower_bnd = 5
upper_bnd = 17 

transw = 0.1

samprate = fs
filtorder = 10*np.round(samprate/lower_bnd)+1

filter_shape = [0, 0, 1, 1, 0, 0]
filter_freqs = [0, lower_bnd*(1-transw), lower_bnd, upper_bnd, upper_bnd*(1+transw), samprate/2]

filterkern = signal.firls(filtorder,filter_freqs,filter_shape,fs=samprate)
hz_filt = np.linspace(0,samprate/2,int(np.floor(len(filterkern)/2)+1))
filterpow = np.abs(scipy.fft.fft(filterkern))**2


## plots
plt.subplot(121)
plt.plot(filterkern)
plt.xlabel('Time points')
plt.title('Filter kernel (firls)')

# plot amplitude spectrum of the filter kernel
plt.subplot(122)
plt.plot(hz_filt,filterpow[:len(hz_filt)],'ks-')
plt.plot(filter_freqs,filter_shape,'ro-')

plt.xlim([0,upper_bnd+40])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Filter gain')
plt.title('Frequency response')
plt.show()

In [ ]:
x_filt1 = signal.filtfilt(filterkern,1,x)

# plot time series
plt.subplot(211)
plt.plot(time_vec, x, label =  'original')
plt.plot(time_vec, x_filt1, label = 'first bandpass filter')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Time Domain')


x_filt1_pwr = np.abs(scipy.fft.fft(x_filt1)/npnts)**2

plt.subplot(212)
plt.plot(hz, pwr_x, label='original')
plt.plot(hz, x_filt1_pwr, label='filtered 1')
plt.title('Frequency domain')
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.subplots_adjust(hspace=1)
plt.xlim([0,40])


In [ ]:

# Define second filter parameter
lower_bnd = 25
upper_bnd = 32

transw = 0.1

frange = [lower_bnd, upper_bnd]

samprate = fs
filtorder = int( 30 * (samprate/frange[0]))

filter_shape = [0, 0, 1, 1, 0, 0]
filter_freqs = [0, lower_bnd*(1-transw), lower_bnd, upper_bnd, upper_bnd*(1+transw), samprate/2]


filterkern = signal.firwin(filtorder,frange,pass_zero = False,fs=samprate)
hz_filt = np.linspace(0,samprate/2,int(np.floor(len(filterkern)/2)+1))
filterpow = np.abs(scipy.fft.fft(filterkern))**2


## plots
plt.subplot(121)
plt.plot(filterkern)
plt.xlabel('Time points')
plt.title('Filter kernel (firls)')

# plot amplitude spectrum of the filter kernel
plt.subplot(122)
plt.plot(hz_filt,filterpow[:len(hz_filt)],'ks-')
plt.plot(filter_freqs,filter_shape,'ro-')

plt.xlim([0,upper_bnd+40])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Filter gain')
plt.title('Frequency response')
plt.xlim([15, 40])
plt.show()


In [ ]:
x_filt2 = signal.filtfilt(filterkern,1,x)

# plot time series
plt.subplot(211)
plt.plot(time_vec, x, label =  'original')
plt.plot(time_vec, x_filt2, label = 'first bandpass filter')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Time Domain')


x_filt2_pwr = np.abs(scipy.fft.fft(x_filt2)/npnts)**2

plt.subplot(212)
plt.plot(hz, pwr_x, label='original')
plt.plot(hz, x_filt2_pwr, label='filtered 1')
plt.title('Frequency domain')
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.subplots_adjust(hspace=1)
plt.xlim([0,40])

In [ ]:
# Add power spectra
x_filt3_pwr = x_filt2_pwr + x_filt1_pwr

plt.plot(hz, pwr_x, label='original')
plt.plot(hz, x_filt3_pwr, label='filtered 1')
plt.title('Frequency domain')
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power')
plt.subplots_adjust(hspace=1)
plt.xlim([0,40])

In [ ]:
# Take ifft of summed power spectra
filt_x = np.real(scipy.fft.ifft(x_filt3_pwr))

plt.subplot(211)
plt.plot(time_vec, filt_x)

plt.subplot(212)
plt.plot(time_vec, x, label = 'original')
plt.plot(time_vec, filt_x, label = 'filtered')

